In [1]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.1 MB/s eta 0:00:00


In [2]:
# --- Alaa's Side ---
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
import hashlib

In [4]:
# Generate Alaa's RSA keys
alaa_key = RSA.generate(2048)
alaa_private = alaa_key
alaa_public = alaa_key.publickey()

In [5]:
# Save Alaa's public key
with open("alaa_pub.pem", "wb") as f:
    f.write(alaa_public.export_key())

In [6]:
# Load Mohamed's public key (exchange manually)
mohamed_public = RSA.import_key(open("mohamed_pub.pem", "rb").read())

In [7]:
# Generate Alaa's random AES part
import os
alaa_random = os.urandom(16)

In [8]:
# Encrypt Alaa's part with Mohamed's public key
encrypted_alaa_random = PKCS1_OAEP.new(mohamed_public).encrypt(alaa_random)
with open("encrypted_alaa_key.bin", "wb") as f:
    f.write(encrypted_alaa_random)

In [9]:
# Load Mohamed's encrypted part
encrypted_mohamed_random = open("encrypted_mohamed_key.bin", "rb").read()

In [10]:
# Decrypt with Alaa's private key
mohamed_random = PKCS1_OAEP.new(alaa_private).decrypt(encrypted_mohamed_random)

In [11]:
# Generate session key
session_key = hashlib.sha256(mohamed_random + alaa_random).digest()[:16]

In [13]:
# Decrypt message
data = open("encrypted_message.bin", "rb").read()
nonce, tag, ciphertext = data[:16], data[16:32], data[32:]
message = AES.new(session_key, AES.MODE_EAX, nonce=nonce).decrypt_and_verify(ciphertext, tag)

In [14]:
# Verify signature
signature = open("signature.bin", "rb").read()
h = SHA256.new(message)
try:
    pkcs1_15.new(mohamed_public).verify(h, signature)
    print("Signature valid.")
except (ValueError, TypeError):
    print("Signature invalid.")

print("Decrypted message:", message.decode())

Signature valid.
Decrypted message: MohamedID202201647
